In [5]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [3]:
directions = ["left", "right", "back"]
all_coordinates = {}
for direction in directions:
    # 비디오 파일 이름들
    input_files = [f"D:\\Pitch\\sample_videos\\{i}_{direction}.mp4" for i in range(1,21)] # 1부터 20까지의 비디오 경로들

In [6]:
for direction in directions:
    # 비디오 파일 이름들
    input_files = [f"D:\\Pitch\\sample_videos\\{i}_{direction}.mp4" for i in range(1,21)] # 1부터 20까지의 비디오 경로들

    for input_file in input_files:
        if not os.path.exists(input_file):
            print(f"{input_file}이 존재하지 않습니다.")
            continue  # 파일이 존재하지 않으면 다음 파일로 넘어감

        # 비디오 파일의 경우 이것을 사용하세요:
        cap = cv2.VideoCapture(input_file)
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # 어깨 관절 좌표화 코드
        right_shoulder_coordinates = [] # 우완투수

        with mp_pose.Pose(
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as pose:

            while cap.isOpened():
                success, image = cap.read()
                if not success:
                    print(f"{input_file}을 처리하는데 실패했습니다.")
                    break

                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = pose.process(image)

                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                image_height, image_width, _ = image.shape

                if results.pose_landmarks:
                    right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    right_shoulder_coordinates.append((right_shoulder.x, right_shoulder.y))

        cap.release()

        all_coordinates[input_file] = right_shoulder_coordinates

        # 팔꿈치 좌표 출력
        for i, coord in enumerate(right_shoulder_coordinates): # 왼손투수의 경우 left_shoulder_coordinates로 바꿔주세요.
            print(f"Frame {i+1}: x = {coord[0] * image_width}, y = {coord[1] * image_height}")

        # 좌표를 데이터프레임에 저장
        df = pd.DataFrame(right_shoulder_coordinates, columns=["x", "y"]) # x_back, x_left, x_right, y_back, y_left, y_right 중 선택

        output_dir = "D:/Pitch/coordinates"
        output_filename = os.path.basename(os.path.splitext(input_file)[0]) + "_shoulder_coordinates.csv"
        output_file_path = os.path.join(output_dir, output_filename)

        df.to_csv(output_file_path, index=False)

D:\Pitch\sample_videos\1_left.mp4을 처리하는데 실패했습니다.
Frame 1: x = 841.3674259185791, y = 872.6395797729492
Frame 2: x = 842.2373628616333, y = 873.3791542053223
Frame 3: x = 842.8490352630615, y = 874.5690536499023
Frame 4: x = 843.1370401382446, y = 874.5529174804688
Frame 5: x = 843.211841583252, y = 874.4917488098145
Frame 6: x = 843.7565016746521, y = 874.4855117797852
Frame 7: x = 844.0517163276672, y = 874.628849029541
Frame 8: x = 844.0719938278198, y = 874.5565223693848
Frame 9: x = 844.0672945976257, y = 874.4183921813965
Frame 10: x = 844.0821003913879, y = 873.9062118530273
Frame 11: x = 844.088408946991, y = 873.7489128112793
Frame 12: x = 844.4561719894409, y = 873.9345932006836
Frame 13: x = 844.827024936676, y = 874.0486907958984
Frame 14: x = 845.0918555259705, y = 874.057674407959
Frame 15: x = 845.1517868041992, y = 874.1281700134277
Frame 16: x = 845.151207447052, y = 874.2356300354004
Frame 17: x = 845.4047083854675, y = 874.3018341064453
Frame 18: x = 846.0994219779968